# Optimizing sequences using a fine-tuned AlphaBind model
**2024-10-21**

## Introduction

Once you have a fine-tuned model of interest, the following tutorial demonstrates how to run optimization to generate sequences to improve their ability for a given affinity-proxy property. Please note **the affinity-proxy is expected to be favorable downhill**, this means the smaller the value, the "better" the proposal. Thus, ensure the sign of your proxy is pointing to `-np.inf`.

**NOTE**: Optimization only works against 1 target at the moment; multi-target optimization may be supported at a later date.

Running optimization will do the following under the hood for "N" steps (AKA generations):
- Given starting sequence(s), propose a mutation(s)
- Evaluate the new sequence against a given target
- Select whether to accept or reject the proposed mutation, based on the relative difference of the affinity-proxy score.

Optimization always works on `sequence_a`. Thus, any proposed changes will always be to the `sequence_a` input.

## Prerequisites

#### Build the `alphabind` Docker image

The following steps depend on the `alphabind` Docker image, which you will need to build yourself due to BioNeMo's licensing restrictions. See the primary [`README.md`](../../../README.md) for detailed instructions of how to build that image.

#### Fine-tune the AlphaBind pre-trained model checkpoint

We **strongly** recommend that you first pre-train the AlphaBind pre-trained checkpoint (`alphabind/models/alphabind_pretrained_checkpoint.pt`) before attempting to perform optimization using this package. Follow our [fine-tuning and inference tutorial](../finetuning_and_inference/tutorial_1_finetuning_alphabind.ipynb) to complete this step before proceeding with this tutorial.

## Customize your config

An example config that accompanies this tutorial is provided in `alphabind/examples/optimization/conf/optimize.env`.

This config specifies environment variables which are read by Click to provision function arguments for `alphabind.models.optimize_seeds`. These environment variables are all prefixed with `ALPHABIND_`, but otherwise correspond exactly to the argument names defined in that module.

For example, the `ALPHABIND_SEED_SEQUENCE` environment variable in `optimize.env` corresponds to the `seed_sequence` argument of the `alphabind.models.optimize_seeds` module. For more details on each argument, consult the provided docstring in the source code. For convenience, we also provided a brief summary of relevant arguments below.

**NOTE:** Several of the config values depend on config values from previous steps (e.g. Tutorial 1). Our provided example configs are all intercompatible, however if you modified any of the paths or other values, you may need to correspondingly update them in the configs for this tutorial.

### Overview of parameters in `optimize.env`

- `ALPHABIND_SEED_SEQUENCE`: The full sequence of the protein of interest. This is the sequence that gets modified. Assigned to `sequence_a` automatically.
- `ALPHABIND_TARGET`: The full sequence of the protein target. This is the sequence that you are optimizing against.
- `ALPHABIND_MUTATION_START_IDX`: 0-indexed start position of the mutation window
- `ALPHABIND_MUTATION_END_IDX`: End position of the mutation window. Is inclusive.
- `ALPHABIND_NUM_SEEDS`:  Number of copies of the seed sequence to optimize.
- `ALPHABIND_GENERATIONS`: Number of proposed steps per sequence. For "N" generations, there will be "N" proposal/acceptance steps attempted.
- `ALPHABIND_TRAINED_MODEL_PATH`: Path in the Docker guest to your already fine-tuned model checkpoint
- `ALPHABIND_OUTPUT_FILE_PATH`: Output path in the Docker guest where results from the final generation will be written.
- `ALPHABIND_SAVE_INTERMEDIATE_STEPS`: Path to a directory in the Docker guest where results from all generations will be written.
- `ALPHABIND_GENERATOR_TYPE`: Sequence mutation strategy to use during optimization.

#### Mutation Window Details

The following config variables control the mutation window that actually receives proposed changes. The three key variables are `ALPHABIND_SEED_SEQUENCE`, `ALPHABIND_MUTATION_START_IDX`, and `ALPHABIND_MUTATION_END_IDX`. Take for example the following values (as taken from the provided `optimize.env`):

```
ALPHABIND_SEED_SEQUENCE=EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSSGGGGSGGGGSGGGGSDIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIKR
ALPHABIND_MUTATION_START_IDX=98
ALPHABIND_MUTATION_END_IDX=107
```

This would mean the window of mutations would look like this: `WGGDGFYAMD`. This would be comparable to assigning `ALPHABIND_SEED_SEQUENCE` to a python string, and slicing it as follows: `ALPHABIND_SEED_SEQUENCE[ ALPHABIND_MUTATION_START_IDX: (ALPHABIND_MUTATION_END_IDX + 1)]`.

## Running Optimization

**NOTE:** This step completes in <5 min. on a `g5.4xlarge` using the provided `optimize.env`. However, it is potentially a multi-hour step if you have modified `optimize.env` to typical production scale. We provide an inline implementation in this notebook for convenience, but you may wish to manually run this step in a terminal emulator (e.g. `tmux`, `screen`) in order to increase resilience of the training job against e.g. temporary network interruptions to a remote instance.

You can run the following to optimize a parent sequence against a target. Using the provided config (`optimize.env`), we optimize against the `Mason et al.` data that is cited and pre-processed in our [fine-tuning and inference tutorial](../finetuning_and_inference/tutorial_1_finetuning_alphabind.ipynb) using the fine-tuned model from that tutorial.

In [1]:
! docker run --rm -it --init --entrypoint python --gpus=all --shm-size=64G --env-file=alphabind/examples/optimization/conf/optimize.env --name=alphabind_optimize -v ./alphabind/examples/data:/mnt/data alphabind:latest -m alphabind.optimizers.optimize_seeds

docker: open alphabind/examples/finetuning_and_optimization/conf/optimize.env: no such file or directory.
See 'docker run --help'.


## Merging optimization candidates from all generations

In the provided example config (`optimize.env`), we specify `ALPHABIND_SAVE_INTERMEDIATE_STEPS`, which causes the optimizer to save candidates from intermediate generations in addition to the final generation. This is often useful when selecting downstream candidates for validation as it both:

- Allows selecting candidates within a desired range of edit distances from the parental sequence _without_ the need to pre-specify a target number of generations to fall within that range at the final generation
- Allows capturing promising intermediate parental candidates within a MCMC thread which were later further optimized by the final generation.

In order to combine results from all generations, we provide a helper script, which can be run as:

In [2]:
! docker run --rm -it --init --entrypoint python --gpus=all --shm-size=64G --env-file=alphabind/examples/optimization/conf/merge.env --name=alphabind_merge -v ./alphabind/examples/data:/mnt/data alphabind:latest -m alphabind.optimizers.merge_all_generations

docker: open alphabind/examples/finetuning_and_optimization/conf/merge.env: no such file or directory.
See 'docker run --help'.


After the step above completes, combined results may be found at `alphabind/examples/data/results/all_unique_candidates.csv` if using our provided example `merge.env` (or otherwise as specified by `ALPHABIND_OUTPUT_FILE_PATH` in that file).